In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
df = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190518.txt')

In [3]:
df.columns = df.columns.str.strip()

Getting the day of the week column

In [4]:
df['DATETIME'] = df['DATE'] + ' ' + df['TIME']
df['DATETIME'] = pd.to_datetime(df['DATETIME'], format = '%m/%d/%Y %H:%M:%S')

In [5]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,00:00:00,REGULAR,7053483,2390505,2019-05-11 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,04:00:00,REGULAR,7053511,2390508,2019-05-11 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,08:00:00,REGULAR,7053534,2390536,2019-05-11 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,12:00:00,REGULAR,7053618,2390596,2019-05-11 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,16:00:00,REGULAR,7053841,2390655,2019-05-11 16:00:00


In [6]:
df['DAY'] = df['DATETIME'].dt.day_name()

In [7]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,DAY
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,00:00:00,REGULAR,7053483,2390505,2019-05-11 00:00:00,Saturday
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,04:00:00,REGULAR,7053511,2390508,2019-05-11 04:00:00,Saturday
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,08:00:00,REGULAR,7053534,2390536,2019-05-11 08:00:00,Saturday
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,12:00:00,REGULAR,7053618,2390596,2019-05-11 12:00:00,Saturday
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,16:00:00,REGULAR,7053841,2390655,2019-05-11 16:00:00,Saturday


Get each every exit and entry delta

In [8]:
df['ENTRY_DELTA'] = df.groupby(['UNIT', 'SCP'])['ENTRIES'].transform(lambda x: x.diff())
df['EXIT_DELTA'] = df.groupby(['UNIT', 'SCP'])['EXITS'].transform(lambda x: x.diff())

In [9]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,DAY,ENTRY_DELTA,EXIT_DELTA
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,00:00:00,REGULAR,7053483,2390505,2019-05-11 00:00:00,Saturday,nan,nan
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,04:00:00,REGULAR,7053511,2390508,2019-05-11 04:00:00,Saturday,28.000,3.000
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,08:00:00,REGULAR,7053534,2390536,2019-05-11 08:00:00,Saturday,23.000,28.000
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,12:00:00,REGULAR,7053618,2390596,2019-05-11 12:00:00,Saturday,84.000,60.000
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,16:00:00,REGULAR,7053841,2390655,2019-05-11 16:00:00,Saturday,223.000,59.000


Eliminate Outliers

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208681 entries, 0 to 208680
Data columns (total 15 columns):
C/A            208681 non-null object
UNIT           208681 non-null object
SCP            208681 non-null object
STATION        208681 non-null object
LINENAME       208681 non-null object
DIVISION       208681 non-null object
DATE           208681 non-null object
TIME           208681 non-null object
DESC           208681 non-null object
ENTRIES        208681 non-null int64
EXITS          208681 non-null int64
DATETIME       208681 non-null datetime64[ns]
DAY            208681 non-null object
ENTRY_DELTA    203829 non-null float64
EXIT_DELTA     203829 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(2), object(10)
memory usage: 23.9+ MB


In [11]:
df = df[df['ENTRY_DELTA'] >= 0]
df = df[df['EXIT_DELTA'] >= 0]

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201377 entries, 1 to 208680
Data columns (total 15 columns):
C/A            201377 non-null object
UNIT           201377 non-null object
SCP            201377 non-null object
STATION        201377 non-null object
LINENAME       201377 non-null object
DIVISION       201377 non-null object
DATE           201377 non-null object
TIME           201377 non-null object
DESC           201377 non-null object
ENTRIES        201377 non-null int64
EXITS          201377 non-null int64
DATETIME       201377 non-null datetime64[ns]
DAY            201377 non-null object
ENTRY_DELTA    201377 non-null float64
EXIT_DELTA     201377 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(2), object(10)
memory usage: 24.6+ MB


Dropped about 3.5% of the data

In [13]:
df.describe()

,ENTRIES,EXITS,ENTRY_DELTA,EXIT_DELTA
count,201377.000,201377.000,201377.000,201377.000
mean,28409445.757,20884766.486,14868.041,2601.821
std,159945204.726,137761910.402,5002549.402,799887.465
min,0.000,0.000,0.000,0.000
25%,332169.000,139050.000,9.000,8.000
50%,2065931.000,1208140.000,69.000,51.000
75%,6464181.000,4418950.000,237.000,165.000
max,2115830563.000,2062389317.000,2055525511.000,317577546.000


In [14]:
df = df[df['ENTRY_DELTA'] < 15000]
df = df[df['EXIT_DELTA'] < 15000]

In [15]:
df.describe()

,ENTRIES,EXITS,ENTRY_DELTA,EXIT_DELTA
count,201367.000,201367.000,201367.000,201367.000
mean,28396012.490,20883277.698,173.043,136.300
std,159872812.511,137763291.660,254.645,231.633
min,0.000,0.000,0.000,0.000
25%,332181.500,139050.000,9.000,8.000
50%,2065955.000,1208144.000,69.000,51.000
75%,6463803.500,4418906.500,237.000,165.000
max,2115830563.000,2062389317.000,3946.000,5406.000


that was only ten entries

In [16]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,DAY,ENTRY_DELTA,EXIT_DELTA
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,04:00:00,REGULAR,7053511,2390508,2019-05-11 04:00:00,Saturday,28.000,3.000
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,08:00:00,REGULAR,7053534,2390536,2019-05-11 08:00:00,Saturday,23.000,28.000
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,12:00:00,REGULAR,7053618,2390596,2019-05-11 12:00:00,Saturday,84.000,60.000
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,16:00:00,REGULAR,7053841,2390655,2019-05-11 16:00:00,Saturday,223.000,59.000
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,20:00:00,REGULAR,7054133,2390702,2019-05-11 20:00:00,Saturday,292.000,47.000


In [17]:
df.describe()

,ENTRIES,EXITS,ENTRY_DELTA,EXIT_DELTA
count,201367.000,201367.000,201367.000,201367.000
mean,28396012.490,20883277.698,173.043,136.300
std,159872812.511,137763291.660,254.645,231.633
min,0.000,0.000,0.000,0.000
25%,332181.500,139050.000,9.000,8.000
50%,2065955.000,1208144.000,69.000,51.000
75%,6463803.500,4418906.500,237.000,165.000
max,2115830563.000,2062389317.000,3946.000,5406.000


get daily traffic figures: going to try to add entry and exit data to get total people entering and exiting on any given day

In [18]:
daily_entries = df.groupby(['STATION', 'LINENAME','DAY']).agg({'ENTRY_DELTA' : 'sum'})

In [19]:
daily_entries.head()

ENTRY_DELTA
STATION LINENAME DAY                  
1 AV    L        Friday      19409.000
                 Monday      16976.000
                 Saturday     5581.000
                 Sunday       4770.000
                 Thursday    19686.000

In [20]:
daily_exits = df.groupby(['STATION', 'LINENAME','DAY']).agg({'EXIT_DELTA' : 'sum'})

Making separate dataframes for only weekends and weekdays

In [21]:
weekends = df[(df.DAY == "Saturday") | (df.DAY == "Sunday")]

In [22]:
weekends.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,DAY,ENTRY_DELTA,EXIT_DELTA
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,04:00:00,REGULAR,7053511,2390508,2019-05-11 04:00:00,Saturday,28.000,3.000
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,08:00:00,REGULAR,7053534,2390536,2019-05-11 08:00:00,Saturday,23.000,28.000
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,12:00:00,REGULAR,7053618,2390596,2019-05-11 12:00:00,Saturday,84.000,60.000
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,16:00:00,REGULAR,7053841,2390655,2019-05-11 16:00:00,Saturday,223.000,59.000
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,20:00:00,REGULAR,7054133,2390702,2019-05-11 20:00:00,Saturday,292.000,47.000


In [23]:
weekdays = df[(df.DAY != "Saturday") & (df.DAY != "Sunday")]

In [24]:
weekdays.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,DAY,ENTRY_DELTA,EXIT_DELTA
12,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/13/2019,00:00:00,REGULAR,7054748,2390918,2019-05-13 00:00:00,Monday,79.000,21.000
13,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/13/2019,04:00:00,REGULAR,7054758,2390920,2019-05-13 04:00:00,Monday,10.000,2.000
14,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/13/2019,08:00:00,REGULAR,7054791,2391016,2019-05-13 08:00:00,Monday,33.000,96.000
15,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/13/2019,08:30:55,REGULAR,7054815,2391066,2019-05-13 08:30:55,Monday,24.000,50.000
16,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/13/2019,12:00:00,REGULAR,7054967,2391242,2019-05-13 12:00:00,Monday,152.000,176.000


In [25]:
weekday_average = weekdays.groupby(['STATION', 'LINENAME']).agg({'ENTRY_DELTA' : 'sum'})

In [26]:
weekday_average.head()

ENTRY_DELTA
STATION       LINENAME             
1 AV          L           94483.000
103 ST        1           57484.000
              6           70313.000
              BC          25846.000
103 ST-CORONA 7          102216.000

In [27]:
weekday_average['ENTRY_DELTA'] = weekday_average['ENTRY_DELTA']/5

In [28]:
weekday_average.head()

ENTRY_DELTA
STATION       LINENAME             
1 AV          L           18896.600
103 ST        1           11496.800
              6           14062.600
              BC           5169.200
103 ST-CORONA 7           20443.200

In [29]:
weekend_average = weekends.groupby(['STATION', 'LINENAME']).agg({'ENTRY_DELTA' : 'sum'})

In [30]:
weekend_average['ENTRY_DELTA'] = weekend_average['ENTRY_DELTA']/2

In [31]:
weekend_average.head()

ENTRY_DELTA
STATION       LINENAME             
1 AV          L            5175.500
103 ST        1            7803.000
              6            5492.000
              BC           2521.000
103 ST-CORONA 7           11792.500

In [32]:
weekend_average.reset_index()

,STATION,LINENAME,ENTRY_DELTA
0,1 AV,L,5175.500
1,103 ST,1,7803.000
2,103 ST,6,5492.000
3,103 ST,BC,2521.000
4,103 ST-CORONA,7,11792.500
5,104 ST,A,621.000
6,104 ST,JZ,9.500
7,110 ST,6,5013.500
8,111 ST,7,7282.500
9,111 ST,A,943.500


In [33]:
weekday_average.reset_index()

,STATION,LINENAME,ENTRY_DELTA
0,1 AV,L,18896.600
1,103 ST,1,11496.800
2,103 ST,6,14062.600
3,103 ST,BC,5169.200
4,103 ST-CORONA,7,20443.200
5,104 ST,A,1592.400
6,104 ST,JZ,2292.400
7,110 ST,6,11168.600
8,111 ST,7,11506.600
9,111 ST,A,2804.400


Grabbing the top ten station from Abi's analysis

In [41]:
top_weekdays = weekday_average.loc[['GRD CNTRL-42 ST','34 ST-HERALD SQ','34 ST-PENN STA','42 ST-PORT AUTH','14 ST-UNION SQ',
                                   'TIMES SQ-42 ST','59 ST COLUMBUS','FULTON ST','FLUSHING-MAIN','PATH NEW WTC']]

In [43]:
top_weekdays.head(50)

ENTRY_DELTA
STATION         LINENAME                 
14 ST-UNION SQ  456LNQRW        31967.200
                LNQR456W        72585.400
34 ST-HERALD SQ BDFMNQRW       119203.600
34 ST-PENN STA  123             37320.600
                123ACE          50635.800
                ACE             87790.200
42 ST-PORT AUTH ACENGRS1237W    13903.200
                ACENQRS1237W    79962.200
59 ST COLUMBUS  1ABCD           15222.200
                ABCD1           59822.600
FLUSHING-MAIN   7               58652.400
FULTON ST       2345ACJZ        65941.000
                ACJZ2345        31397.600
                G                5962.800
GRD CNTRL-42 ST 4567S          161473.800
PATH NEW WTC    1               59408.400
TIMES SQ-42 ST  1237ACENQRS     16940.400
                1237ACENQRSW    66043.400
                ACENQRS1237W    16694.600

In [44]:
top_weekdend = weekend_average.loc[['GRD CNTRL-42 ST','34 ST-HERALD SQ','34 ST-PENN STA','42 ST-PORT AUTH','14 ST-UNION SQ',
                                   'TIMES SQ-42 ST','59 ST COLUMBUS','FULTON ST','FLUSHING-MAIN','PATH NEW WTC']]

Turns out there are multiple stations with the same name... but looking at the line information, it really looks like
it's one station and the lines information is garbled. 